In [1]:
import numpy as np
import pandas as pd
from my_utils import get_df_info
from tqdm import tqdm.notebook
import rapidfuzz
import itertools

In [2]:
df_emp = pd.read_parquet('aim-2024-orcs/employees.parquet', engine='pyarrow')
df_emp.head()

,name,surname,fathername,gender,birthdate,inn,passport
0,наталия,фролова,игоревна,ж,1985-07-31,None,4748233001
1,раиса,фонарева,дмитриевна,ж,1987-04-04,554480956011,2690351333
2,надежда,филатова,павловна,ж,1974-09-25,451793060934,4134295167
3,сергей,володин,алексеевич,м,1964-07-22,None,2295582551
4,дмитрий,чеканов,геннадьевич,м,1963-11-30,261235545350,5849888033


In [3]:
df_orc = pd.read_parquet('aim-2024-orcs/orcs.parquet', engine='pyarrow')
df_orc.head()

,name,surname,fathername,gender,birthdate,inn,passport
0,артавазд,пикта,волрьевич,м,1969-01-24,<NA>,<NA>
1,фамоида,каршибаева,димитриевна,ж,1995-02-26,854523248292,1574350894
2,анна,селиванова,сергеевна,ж,1988-05-06,<NA>,7331985468
3,александр,бояринцев,андреевич,м,1995-10-07,<NA>,5666222340
4,фатхидин,романеко,гаджибахмудович,м,1977-03-04,181559945061,6632447607


In [4]:
df_emp_info = get_df_info(df_emp)
df_orc_info = get_df_info(df_orc)

In [5]:
df_emp_info

,dtype,nunique,example1,example2,zero,nan,empty_str,vc_max,vc_max_prop,trash_score
name,object,41657,наталия,раиса,-1,n: 0.0,-1,александр,0.037,0.0
surname,object,217368,фролова,фонарева,-1,n: 0.0,-1,иванова,0.003,0.0
fathername,object,67217,игоревна,дмитриевна,-1,n: 0.0,-1,александровна,0.043,0.0
gender,object,2,ж,м,-1,-1,-1,ж,0.513,-1
birthdate,object,17798,1985-07-31,1987-04-04,-1,-1,-1,1981-11-26,0.0,-1
inn,object,637770,554480956011,451793060934,-1,n: 0.37,-1,00990643763,0.0,0.37
passport,object,998643,4748233001,2690351333,-1,n: 0.013,-1,9334394331,0.0,0.013


Видно, что за исключением колонок `inn` и `passport` данные практически без пропусков. В колонке `passport` 1.3% пропусков, а в `inn` уже целых 37% пропусков.

In [6]:
df_orc_info

,dtype,nunique,example1,example2,zero,nan,empty_str,vc_max,vc_max_prop,trash_score
name,string,23359,артавазд,фамоида,-1,-1,-1,александр,0.011,-1
surname,string,38712,пикта,каршибаева,-1,-1,-1,кузнецова,0.001,-1
fathername,string,27430,волрьевич,димитриевна,-1,n: 0.0,e: 0.0,александровна,0.014,0.0
gender,string,2,м,ж,-1,-1,-1,м,0.705,-1
birthdate,string,13994,1969-01-24,1995-02-26,-1,n: 0.07,-1,1970-05-29,0.0,0.07
inn,string,20525,854523248292,181559945061,-1,n: 0.569,-1,100107370630,0.0,0.569
passport,string,31470,1574350894,7331985468,-1,n: 0.339,-1,1000373865,0.0,0.339


В таблице орков пропусков больше, и они распределены по тем же колонкам.

In [7]:
df = df_emp

Заполним пропуски в таблицах случайными валидными значениями из колонок.

In [8]:
def fill_na_with_random(df, column):
    """
    Заполняет пропуски (NaN) в указанной колонке случайными значениями из уникальных значений этой же колонки.
    Если уникальных значений меньше, чем количество NaN, уникальные значения повторяются.
    
    Параметры:
    - df: pd.DataFrame — исходный датафрейм.
    - column: str — название колонки, где нужно заполнить пропуски.
    """
    unique_values = df[column].dropna().unique()

    num_missing = df[column].isna().sum()
    
    if num_missing > 0:
        random_fill = np.random.choice(unique_values, size=num_missing, replace=True)
        df.loc[df[column].isna(), column] = random_fill
    return df

In [9]:
for col in df:
    df = fill_na_with_random(df, col)

In [10]:
for col in df_orc:
    df_orc_filled = fill_na_with_random(df_orc, col)

In [11]:
def top_k_argmin_rows(matrix, k):
    """
    Находит k минимальных индексов для каждой строки матрицы.
    
    Параметры:
    - matrix: np.ndarray — входная матрица (2D).
    - k: int — количество минимальных индексов для каждой строки.
    
    Возвращает:
    - np.ndarray: массив с k минимальными индексами для каждой строки.
    """
    partitioned_indices = np.argpartition(matrix, k, axis=1)[:, :k]
    sorted_indices = np.array([
        row[np.argsort(matrix[i, row])]
        for i, row in enumerate(partitioned_indices)
    ])
    return sorted_indices

In [12]:
def find_intersections(sets, k=3):
    """
    Находит множества, полученные пересечением любых k из заданных множеств.

    Параметры:
    - sets: list[set] — список входных множеств.
    - k: int — количество множеств в пересечении, k in [2, 3, 4]

    Возвращает:
    - list[set]: список уникальных пересечений.
    """
    intersections = set()
    if k == 4:
        for a, b, c, d in itertools.combinations(sets, 4):
            intersection = a & b & c & d
            intersections.add(frozenset(intersection))
    elif k == 3:
        for a, b, c in itertools.combinations(sets, 3):
            intersection = a & b & c
            intersections.add(frozenset(intersection))
    elif k == 2:
        for a, b in itertools.combinations(sets, 2):
            intersection = a & b
            intersections.add(frozenset(intersection))
    else:
        raise ValueError("not supported k value")
    return [set(s) for s in intersections]

Для приближенного поиска ближайших соседей воспользуемся кластеризацией. Выберем случайно для колонок из списка `["surname", "birthdate", "inn", "passport"]` по `size=1000` центроидов. (выбраны из рассуждений, что по ним наиболее информативно можно поделить на кластеры) 

In [13]:
cols = ["surname", "birthdate", "inn", "passport"]

Каждый центроид порождает кластер (в своей колонке). Выберем ниже для каждого центроида `k=30000` элементов в кластер. Элементы берутся как ближайшие к центроиду. У нас для каждой колонки по `size=1000` центроидов, соответственно близость определяется метрикой Левенштейна на строках из колонки.

Таким образом, получим `len(cols) * size` = 4*1000 кластеров.

In [14]:
def centroid_col_dict_func(df, cols):
    """
    Функция формирует по заданным колонкам из cols
    словарь, хранящий информацию о центроидах кластеров
    и элементах, лежащих в этих кластерах
    
    Параметры:
    - df: pd.DataFrame — список работников
    - cols: list - список колонок, по которым нужно кластеризовать

    Возвращает:
    - dict(col: tuple(numpy.ndarray, numpy.ndarray))

    P.S. Все метрические свойства вычисляются по метрике Левенштейна
    """
    distance_col = {}
    for col in tqdm(cols, total=len(cols)):
        rand_ind = np.random.randint(len(df), size=1000)
        distance_col[col] = (
            rand_ind,
            rapidfuzz.process.cdist(
                df[col].iloc[rand_ind].astype(str).to_numpy().reshape(-1),
                df[col].astype(str).to_numpy().reshape(-1),
                scorer=rapidfuzz.distance.Levenshtein.distance,
            ),
        )
    centroid_col_dict = {}
    for col in tqdm(cols, total=len(cols)):
        centroid_col_dict[col] = (
            distance_col[col][0],
            top_k_argmin_rows(distance_col[col][1], k=30000),
        )
    return centroid_col_dict

In [15]:
centroid_col_dict = centroid_col_dict_func(df, cols)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:16<00:00, 34.23s/it]


In [16]:
print(f"{type(centroid_col_dict) = },                         {len(centroid_col_dict) = }")
print(f"{type(centroid_col_dict['inn']) = },                 {len(centroid_col_dict['inn']) = }")
print(f"{type(centroid_col_dict['inn'][0]) = },      {centroid_col_dict['inn'][0].shape = }")
print(f"{type(centroid_col_dict['inn'][1]) = },      {centroid_col_dict['inn'][1].shape = }")

type(centroid_col_dict) = <class 'dict'>,                         len(centroid_col_dict) = 4
type(centroid_col_dict['inn']) = <class 'tuple'>,                 len(centroid_col_dict['inn']) = 2
type(centroid_col_dict['inn'][0]) = <class 'numpy.ndarray'>,      centroid_col_dict['inn'][0].shape = (1000,)
type(centroid_col_dict['inn'][1]) = <class 'numpy.ndarray'>,      centroid_col_dict['inn'][1].shape = (1000, 30000)


Полученный словарь `centroid_col_dict` в качестве ключей имеет названия колонок, значения - кортежи. В кортеже первый элемент - массив NumPy, в котором хранятся индексы центроидов, второй элемент - двумерный массив NumPy, который хранит для каждого центроида индексы `k=30000` ближайших соседей.

Ниже считаем расстояния (для каждой колонки) от всех орков до всех центроидов.

In [18]:
def dist_orc_col(df_orc, df_centr, col):
    """
    Функция вычисляет расстояния от каждого орка
    до всех центроидов из df_centr по колонке col
    
    Параметры:
    - df_orc: pd.DataFrame — список орков
    - df_centr: pd.DataFrame — список центроидов (центроид - некоторый работник)
    - col: str - название колонки, по которой считается расстояние

    Возвращает:
    - np.ndarray: массив с расстояниями Левенштейна от орков до центроидов
    """
    result = rapidfuzz.process.cdist(
        df_orc[col].astype(str).to_numpy().reshape(-1),
        df_centr[col].astype(str).to_numpy().reshape(-1),
        scorer=rapidfuzz.distance.Levenshtein.distance,
    )
    return result

In [19]:
def orc_nearest_centr_col_func(df, df_orc, cols, centroid_col_dict):
    """
    Функция вычисляет для каждого орка ближайший центроид
    по каждой колонке
    
    Параметры:
    - df: pd.DataFrame — список работников
    - df_orc: pd.DataFrame — список орков
    - cols: list - список названий колонок, по которым
        ищется центроид
    - centroid_col_dict: dict(col: tuple(numpy.ndarray, numpy.ndarray)) —
        хранит информацию по каждой колонке о центроидах и элементах,
        которые принадлежат соответствующему центроиду кластеру,
        в numpy.ndarray хранятся индексы

    Возвращает:
    - orc_nearest_centr_col: dict(col: numpy.ndarray) —
        хранит информацию по каждой колонке о ближайшем
        центроиде для каждого орка
    """
    orc_dist_col = {}
    for col in tqdm(cols):
        orc_dist_col[col] = dist_orc_col(df_orc, df.iloc[centroid_col_dict[col][0]], col)
    orc_nearest_centr_col = {}
    for col in tqdm(cols, total=len(cols)):
        orc_nearest_centr_col[col] = top_k_argmin_rows(orc_dist_col[col], k=1)[:,0]
    return orc_nearest_centr_col

Выделим для каждого орка ближайший центроид (на самом деле набор центроидов, соответствующих каждой колонке).

In [20]:
orc_nearest_centr_col = orc_nearest_centr_col_func(df, df_orc, cols, centroid_col_dict)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.27it/s]


Представленная ниже функция находит приближенных ближайших соседей и расстояния до них. Поиск проходит следующим образом.

Сначала отбираются кандидаты в ближайшие соседи. Для каждого орка имеем 4 кластера (по каждой колонке). Возьмем всевозможные пересечения двух кластеров как множеств, а затем возьмем объединение получившихся пересечений. Таким образом, отберем такие объекты, которые близки к нашему орку хотя бы по двум признакам.

В среднем в таком случае будет оставаться порядка $10^2$ объектов.

Затем уже среди данных ищем ближайшего к нашему орку, причем под расстоянием подразумеваем сумму расстояний Левенштейна по всем колонкам.

In [21]:
def ANN(df, df_orc, centroid_col_dict, orc_nearest_centr_col):
    """
    Находит для каждого орка из df_orc индекс ближайшего
    соседа из df и расстояние до него. Поиск осуществляется
    с помощью заготовленных заранее кластеров с центроидами.

    Параметры:
    - df: pd.DataFrame — список работников, из которых хотим найти орков
    - df_orc: pd.DataFrame — список орков
    - centroid_col_dict: dict(col: tuple(numpy.ndarray, numpy.ndarray)) —
        хранит информацию по каждой колонке о центроидах и элементах,
        которые принадлежат соответствующему центроиду кластеру,
        в numpy.ndarray хранятся индексы
    - orc_nearest_centr_col: dict(col: numpy.ndarray) —
        хранит информацию по каждой колонке о ближайшем
        центроиде для каждого орка

    Возвращает:
    - result_index: numpy.ndarray: индексы ближайших соседей для каждого орка
    - result_value: numpy.ndarray: расстояние до ближайшего соседа

    P.S. Все метрические свойства вычисляются по метрике Левенштейна
    """
    result_index = np.zeros(len(df_orc), dtype="int64")
    result_value = np.zeros(len(df_orc), dtype="int64")
    cols = ["surname", "birthdate", "inn", "passport"]
    tmp = {}
    for col in tqdm(cols, total=len(cols)):
        tmp[col] = centroid_col_dict[col][1][orc_nearest_centr_col[col], :]
    for i in tqdm(range(len(df_orc))):
        set_candidate = set.union(
            *find_intersections([set(tmp[col][i]) for col in cols], k=2)
        )
        list_candidate = list(set_candidate)
        sum = np.zeros(len(list_candidate), dtype="uint32")
        for col in df.columns:
            sum += rapidfuzz.process.cdist(
                [df_orc[col].iloc[i]],
                df[col].iloc[list_candidate],
                scorer=rapidfuzz.distance.Levenshtein.distance,
            )[0]
        result_value[i] = np.min(sum)
        result_index[i] = list_candidate[np.argmin(sum)]
    return result_index, result_value

In [22]:
result_index, result_value = ANN(df, df_orc_filled, centroid_col_dict, orc_nearest_centr_col)

100%|████████████████████████████████████████████████████████████████████████████| 47633/47633 [27:10<00:00, 29.22it/s]


Посмотрим, как распределены получившиеся расстояния до вычисленных ближайших соседей.

In [23]:
print("Array =", result_value)
print("Length = ", result_value.shape[0])
print("Measures of Dispersion")
print("Minimum =", np.amin(result_value))
print("Maximum =", np.amax(result_value))
print("Range =", np.ptp(result_value))
print("Standard Deviation =", np.std(result_value))
print(f"mean = {np.mean(result_value)}")
print(f"median = {np.median(result_value)}")
print(f"10% quantile = {np.quantile(result_value, 0.1)}")
print(f"20% quantile = {np.quantile(result_value, 0.2)}")
print(f"25% quantile = {np.quantile(result_value, 0.25)}")
print(f"50% quantile = {np.quantile(result_value, 0.5)}")

Array = [29 30 11 ... 28 30 29]
Length =  47633
Measures of Dispersion
Minimum = 0
Maximum = 50
Range = 50
Standard Deviation = 10.377760559269596
mean = 25.7810131631432
median = 30.0
10% quantile = 9.0
20% quantile = 12.0
25% quantile = 19.0
50% quantile = 30.0


Видим, что многие кандидаты в орки имеют большие расстояния от реальных орков. Поэтому будет разумно взять не всю получившуюся выборку, а только ее часть, у которой расстояния наименьшие.

Методом пристального взгляда и нескольких попыток отправок на kaggle было выяснено, что значение порога, равное 23, является не самым плохим.

In [24]:
(result_value <= 23).sum()

13340

In [25]:
round((result_value <= 23).sum()/result_value.shape[0],4)

0.2801

Итого, в кандидаты записываем 28% из вычисленной выборки, самый отдаленный сосед от орка находится на расстоянии 23 (метрика Левенштейна, сумма по колонкам).

In [26]:
answer = result_index[result_value <= 23]

In [27]:
res = pd.DataFrame({
    'orig_index': df.iloc[answer].index.astype(np.uint64),
}).reset_index(names='id')
res.to_parquet('submission.parquet', index=False)
res

,id,orig_index
0,0,145079
1,1,883225
2,2,671987
3,3,294944
4,4,223716
...,...,...
13335,13335,431291
13336,13336,319502
13337,13337,298370
13338,13338,710370


### Как еще можно улучшить поиск.
Далее перечислены некоторые методы, которые позволили бы (возможно) улучшить скор. (На реализацию которых у меня, к сожалению, не хватило времени :( )
1. В начале ноутбука было подмечено, что есть категория, в которой есть не малая доля NaN. Также было подмечено, что в таблице с орками заметно больше содержание отсутствующих значений. Эти наблюдения можно использовать для более тонкого предсказания, является ли работник орком. Другими словами, более сложная система обработки отсутствующих данных могла бы возможно значительно увеличить целевую метрику.
2. Возможно более хороший результат бы принес бы анализ различных метрик для сравнения строк. Вполне вероятно, что для разных сравнений могут хорошо работать разные метрики, а не только расстояние Левенштейна. Также возможно помогла бы настройка весов по колонкам при суммировании расстояния Левенштейна.